In [6]:
import csv 

csv_path = 'csv classification with study uid and arm head legs results'
with open(csv_path, 'r') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    dataset = []
    for row in reader:
        dataset.append(row)
del dataset[0] #remove header !

In [7]:
csv_path_ahl = 'ahl nifti results'
with open(csv_path_ahl, 'r') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    dataset_ahl = []
    for row in reader:
        dataset_ahl.append(row)
del dataset_ahl[0] #remove header !


csv_path_gained = 'gained nifti results'
with open(csv_path_gained, 'r') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    dataset_gained = []
    for row in reader:
        dataset_gained.append(row)
del dataset_gained[0] #remove header !

1956


In [ ]:
dataset_reunite = dataset_ahl+dataset_gained 

for row in dataset : 
    study_uid = row[1]
    for patient in dataset_reunite : 
        if study_uid == patient[2] : 
            row.append(patient[5]) #ajouter chemin CT nifti 

In [ ]:
directory = ''
filename = 'classification_dataset.csv'

with open(os.path.join(directory, filename), 'w') as csv_file : 
    csv_writer = csv.writer(csv_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(["PATIENT_ID", "STUDY_UID", "NIFTI_CT", "RIGHT_ARM", "LEFT_ARM", "HEAD", "LEGS"])
    for serie in dataset: 
        #serie = [path_1, modal_1, path_2, modal_2, study_uid, type_, patient_id, nifti_CT, nifti_PET, nifti_MASK]
        csv_writer.writerow([   ])

In [8]:
dataset[0][3]

'/media/oncopole/d508267f-cc7d-45e2-ae24-9456e005a01f/NIFTI/AHL_NIFTI/13011101331015/pet0/1.2.124.113532.10.149.172.5.20131107.93455.7725981_nifti_CT.nii'

In [10]:
import SimpleITK as sitk 
for row in dataset : 
    for i in range(3,6):
        img = sitk.ReadImage(row[i])

In [4]:
for row in dataset : 
    for i in range(3,6):
        path = row[i]
        new_name = path.replace('DDArchive', 'd508267f-cc7d-45e2-ae24-9456e005a01f/NIFTI')
        row[i] = new_name
print(dataset[0])

['13011101331015', 'pet0', '1.2.124.113532.10.149.172.5.20131107.93455.7725981', '/media/oncopole/d508267f-cc7d-45e2-ae24-9456e005a01f/NIFTI/AHL_NIFTI/13011101331015/pet0/1.2.124.113532.10.149.172.5.20131107.93455.7725981_nifti_CT.nii', '/media/oncopole/d508267f-cc7d-45e2-ae24-9456e005a01f/NIFTI/AHL_NIFTI/13011101331015/pet0/1.2.124.113532.10.149.172.5.20131107.93455.7725981_nifti_PT.nii', '/media/oncopole/d508267f-cc7d-45e2-ae24-9456e005a01f/NIFTI/AHL_NIFTI/13011101331015/pet0/1.2.124.113532.10.149.172.5.20131107.93455.7725981_nifti_mask_3d.nii', 'ahl']


In [5]:
directory = '/media/oncopole/DD 2To/SEGMENTATION'
filename = 'SEG_NIFTI_PATH_V6.csv'

with open(os.path.join(directory, filename), 'w') as csv_file : 
    csv_writer = csv.writer(csv_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(["PATIENT_ID", "PET", "STUDY_UID", "NIFTI_CT", "NIFTI_PET", "NIFTI_MASK", "STUDY"])
    for serie in dataset: 
        #serie = [path_1, modal_1, path_2, modal_2, study_uid, type_, patient_id, nifti_CT, nifti_PET, nifti_MASK]
        csv_writer.writerow([serie[0], serie[1], serie[2], serie[3], serie[4], serie[5], serie[6]])

In [9]:
import SimpleITK as sitk 
import numpy as np 
path = '/media/oncopole/83c5223d-7a01-4ed0-b268-b877a7da96e2/REMARC_NIFTI/11009101056032/V0/1.2.840.113619.2.290.3.180357632.682.1327904939.426_nifti_mask_3d.nii'
img = sitk.ReadImage(path)
print(img.GetSize())
array =  sitk.GetArrayFromImage(img)
print(array.shape)
print(np.unique(array))

(256, 256, 299)
(299, 256, 256, 38)
[0 1]
